In [2]:
import numpy as np
from tensorflow.keras.applications.xception import preprocess_input
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_auc_score
)

In [3]:
IMG_SIZE = (224, 224)
# Load the VGG16 model without the top layer (for feature extraction)
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

In [4]:
base_path = 'test_zhucchini_cucumber/'

In [6]:
# load dataset
df = pd.read_csv(f"{base_path}embedded_dataset.csv")

In [7]:
def transform_image(image_path, img_size):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img)
    return img_array

# Function to extract features using VGG16
def extract_features(img):
    img = tf.image.resize(img, (224, 224))
    img = preprocess_input(img.numpy())
    img = np.expand_dims(img, axis=0)
    features = base_model.predict(img)
    return features.squeeze()

# Function to load and preprocess image data and extract deep features
def load_image_data(df):
    image_data = []
    labels = []

    for i in range(len(df)):
        image = df['Image'][i].split('/')[-1]
        label = df['Class'][i]
        image_path = f"{base_path}{label}/{image}"
        # Transform the image
        normalized_img = transform_image(image_path, IMG_SIZE)
        # Extract features using the pre-trained model
        features = extract_features(tf.convert_to_tensor(normalized_img))
        image_data.append(features)
        labels.append(label)

    return np.array(image_data), np.array(labels)

In [8]:
X,y = load_image_data(df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model on the extracted features
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

y_proba = log_reg.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_proba)

print("Accuracy:", accuracy)
print("Precision (macro):", precision)
print("Recall (macro):", recall)
print("F1-score (macro):", f1)
print("ROC-AUC:", auc_score)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7733050847457628
Precision (macro): 0.7695405620169038
Recall (macro): 0.7733166077609098
F1-score (macro): 0.7706271601349787
ROC-AUC: 0.8384456205248412

Classification Report:
              precision    recall  f1-score   support

    cucumber       0.82      0.77      0.80       269
    zucchini       0.72      0.77      0.75       203

    accuracy                           0.77       472
   macro avg       0.77      0.77      0.77       472
weighted avg       0.78      0.77      0.77       472

